In [1]:
import pandas as pd

In [2]:
!git clone https://github.com/IliyaZaslacskii/Hybrid-station_1

Cloning into 'Hybrid-station_1'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 11 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (11/11), 6.92 MiB | 7.68 MiB/s, done.


In [3]:
path = '/content/Hybrid-station_1/DATA/tariff_2022.xlsx'

grid_payment = pd.read_excel(
    path,
    skiprows=1,
    decimal=','
).rename(columns={'Unnamed: 0' : 'regions'})

grid_payment = grid_payment.set_index('regions')
grid_payment.index = grid_payment.index.str.rstrip()

# # #

sales_payment = pd.read_excel(
    path,
    sheet_name=1,
    decimal=',',
    names=['Регион', 'ГП', '<.670', '>.670', '>10']
)

sales_payment = sales_payment.set_index('Регион')
sales_payment.index = sales_payment.index.str.rstrip()

# # #

prices = pd.read_excel(
    path,
    sheet_name=2,
    index_col=0,
    parse_dates=True
)

# # #

power = pd.read_excel(
    path,
    sheet_name=3,
    index_col=0,
    parse_dates=True
)

# # #

peaks = pd.read_excel(
    path,
    sheet_name=5,
    index_col=0,
    parse_dates=True
)

# # #

working_days = pd.DataFrame(index=prices.index)
working_days['year'] = working_days.index.year
working_days['month'] = working_days.index.month
working_days['dummy'] = 0

working_days = working_days.groupby(['year', 'month']).count().loc[2022]

In [4]:
# Доступные регионы
peaks.columns

Index(['Алтай Республика', 'Алтайский  край', 'Астраханская область',
       'Белгородская область', 'Брянская  область', 'Владимирская область',
       'Волгоградская область', 'Вологодская область', 'Воронежская область',
       'Дагестан Республика', 'Забайкальский край', 'Ивановская область',
       'Ингушетия Республика', 'Иркутская  область',
       'Кабардино-Балкарская Республика', 'Калужская  область',
       'Карачаево-Черкесская Республика', 'Кемеровская область',
       'Кировская  область', 'Костромская область', 'Краснодарский край',
       'Красноярский край', 'Крым республика', 'Курганская область',
       'Курская область', 'Липецкая  область', 'Москва', 'Мурманская область',
       'Нижегородская область', 'Новгородская область',
       'Новосибирская область', 'Омская область', 'Оренбургская область',
       'Орловская область', 'Пензенская область', 'Пермский  край',
       'Псковская  область', 'Республика Башкортостан', 'Республика Бурятия',
       'Республика Кал

In [10]:
def cost_calc_3(region, load, voltage_class='ВН', max_power='>10', K_energy=1, K_power=1):
    """
    Расчет стоимости электроэнергии по 3-й ЦК
    
    Parameters
    -----------
        region : str
            имя региона
        load : pandas dataframe (index -> datetime64 за 2022 год!)
            массив электропотребления в МВт*ч
        voltage_class : str
            класс напряжения (ВН, СН II, СН I, НН)
        max_power : str
            максимальная мощность эн.прин. энергоуст. потребителя (<.670, >.670, >10)
        K_energy : float
            множитель тарифной ставки на ЭЭ
        K_power : float
            множитель тарифной ставки на "генераторную" мощность
    
    Returns
    -----------
        total_cost : float
            годовые затраты в рублях
        df : pandas dataframe
            подробная таблица с промежуточными расчетами
    """
    
    peak_hours = peaks[[region]]
    peak_hours = peak_hours.set_index(region)
    peak_hours.index.name = 'timestamp'
    peak_hours['peak'] = 1

    df = prices[[region]].join(peak_hours).fillna(0)
    df = df.rename(columns={region : 'price'})
    df['peak'] = df['peak'].astype(int)

    df = df.join(power[[region]]).fillna(method='ffill')
    df = df.rename(columns={region : 'power'})

    transport = grid_payment.loc[region, voltage_class]
    sales = sales_payment.loc[region, max_power] * 10 

    try:
        df['price'] += transport + float(sales)
    except:
        df['price'] += transport + float(sales[0])
    df['price'] += 1.681 + 2.182 + 0.393
    
    df = df.join(load)

    # # #

    df['price'] = df['price'] * K_energy
    df['power'] = df['power'] * K_power

    # # #

    df['year'] = df.index.year
    df['month'] = df.index.month

    df['energy_cost'] = df['price'] * df['load']
    df['power_cost']  = df['peak'] * df['power'] * df['load']

    energy_cost = df.groupby(['month', 'year'])[['energy_cost']].sum()
    power_cost = df[df['power_cost'] > 0].groupby(['month', 'year'])[['power_cost']].sum()
    power_cost = power_cost.join(working_days)
    power_cost['power_cost_mean'] = power_cost['power_cost'] / power_cost['dummy']

    total_cost = energy_cost['energy_cost'].sum() + power_cost['power_cost_mean'].sum()

    return total_cost, df

**Пример расчета**

In [11]:
load = pd.DataFrame(index=pd.date_range(start='2022-01-01 00:00:00', end='2022-12-31 23:00:00', freq='1H'))
load['load'] = 1

total_cost, df = cost_calc_3('Кемеровская область', load, 'НН', '<.670')

print(total_cost)
display(df)

48840765.04003901


,price,peak,power,load,year,month,energy_cost,power_cost
timestamp,,,,,,,,
2022-01-01 00:00:00,5187.856,0,808095.25,1,2022,1,5187.856,0.0
2022-01-01 01:00:00,5187.596,0,808095.25,1,2022,1,5187.596,0.0
2022-01-01 02:00:00,5187.876,0,808095.25,1,2022,1,5187.876,0.0
2022-01-01 03:00:00,5178.746,0,808095.25,1,2022,1,5178.746,0.0
2022-01-01 04:00:00,5155.536,0,808095.25,1,2022,1,5155.536,0.0
...,...,...,...,...,...,...,...,...
2022-12-31 19:00:00,5807.616,0,707693.77,1,2022,12,5807.616,0.0
2022-12-31 20:00:00,5768.346,0,707693.77,1,2022,12,5768.346,0.0
2022-12-31 21:00:00,5684.576,0,707693.77,1,2022,12,5684.576,0.0
